# Danny's Pizza — Dataset Scaffold (OscarF Datasets generator)

> Add blockquote



**Created:** 2025-08-22 00:39

This notebook sets up the **small tables** in-notebook and expects the **large order tables** to be loaded from CSV/XLS files.
It strictly follows the schema from class:

- `pizza_names(pizza_id INT, pizza_name TEXT)`
- `pizza_toppings(topping_id INT, topping_name TEXT)`
- `pizza_recipes(pizza_id INT, toppings TEXT)` where `toppings` is a comma-separated list of `topping_id`s
- `runners(runner_id INT, registration_date DATE)`
- `customer_orders(order_id INT, customer_id INT, pizza_id INT, exclusions VARCHAR(4), extras VARCHAR(4), order_date TIMESTAMP)`
- `runner_orders(order_id INT, runner_id INT, pickup_time VARCHAR(19), distance VARCHAR(7), duration VARCHAR(10), cancellation VARCHAR(23))`



In [2]:
import os
# Usar la carpeta local donde está el notebook
base_path = os.getcwd() + os.sep
print(f"Usando base_path local: {base_path}")

Usando base_path local: c:\Users\Anonimo\OneDrive\Documentos\Python\IA\Week 2\Danny's Pizza\


In [3]:
import pandas as pd

## Create Small Tables (Dims)

We define a **minimal, realistic** pizza catalog.


In [4]:
# --- pizza_names ---
pizza_names = pd.DataFrame({
    'pizza_id': [1, 2, 3, 4, 5, 6, 7],
    'pizza_name': [
        'Margherita',        # 1
        'Vegetarian',        # 2
        'Meat Lovers',       # 3
        'BBQ Chicken',       # 4
        'Hawaiian',          # 5
        'Pepperoni',         # 6
        'Vegan Veggie'       # 7  <-- vegan option (no cheese by default)
    ]
})
pizza_names.to_csv(base_path+'pizza_names.csv', index=False)
pizza_names

,pizza_id,pizza_name
0,1,Margherita
1,2,Vegetarian
2,3,Meat Lovers
3,4,BBQ Chicken
4,5,Hawaiian
5,6,Pepperoni
6,7,Vegan Veggie


In [5]:
# --- pizza_toppings ---
# Concise list so students can reason about extras/exclusions clearly
pizza_toppings = pd.DataFrame({
    'topping_id': list(range(1, 16)),
    'topping_name': [
        'Tomato Sauce',  # 1
        'Mozzarella',    # 2
        'Mushroom',      # 3
        'Onion',         # 4
        'Bell Pepper',   # 5
        'Olives',        # 6
        'Pepperoni',     # 7
        'Bacon',         # 8
        'Beef',          # 9
        'Chicken',       # 10
        'Pineapple',     # 11
        'BBQ Sauce',     # 12
        'Jalapeno',      # 13
        'Fresh Basil',   # 14
        'Garlic'         # 15
    ]
})
pizza_toppings.to_csv(base_path+'pizza_toppings.csv', index=False)
pizza_toppings

,topping_id,topping_name
0,1,Tomato Sauce
1,2,Mozzarella
2,3,Mushroom
3,4,Onion
4,5,Bell Pepper
5,6,Olives
6,7,Pepperoni
7,8,Bacon
8,9,Beef
9,10,Chicken


In [6]:
# --- pizza_recipes ---
# Define base recipes as comma-separated topping_id strings (order does not matter)
recipes_map = {
    1: [1,2,14],               # Margherita: sauce, mozzarella, basil
    2: [1,2,3,4,5,6],          # Vegetarian: sauce, mozzarella, mushroom, onion, bell pepper, olives
    3: [1,2,7,8,9],            # Meat Lovers: sauce, mozzarella, pepperoni, bacon, beef
    4: [1,2,10,12],            # BBQ Chicken: sauce, mozzarella, chicken, bbq sauce
    5: [1,2,11,6],             # Hawaiian: sauce, mozzarella, pineapple, olives
    6: [1,2,7],                # Pepperoni: sauce, mozzarella, pepperoni
    7: [1,3,4,5,6,15]          # Vegan Veggie: sauce, mushroom, onion, bell pepper, olives, garlic (no cheese)
}

pizza_recipes = pd.DataFrame({
    'pizza_id': list(recipes_map.keys()),
    'toppings': [','.join(map(str, v)) for v in recipes_map.values()]
})
pizza_recipes.to_csv(base_path+ 'pizza_recipes.csv', index=False)
pizza_recipes

,pizza_id,toppings
0,1,"1,2,14"
1,2,"1,2,3,4,5,6"
2,3,"1,2,7,8,9"
3,4,"1,2,10,12"
4,5,"1,2,11,6"
5,6,"1,2,7"
6,7,"1,3,4,5,6,15"


In [7]:
# --- runners ---

dates = pd.date_range('2021-01-03', periods=15, freq='7D')
runners = pd.DataFrame({
    'runner_id': range(1, 16),
    'registration_date': dates.date
})
runners.to_csv(base_path+'runners.csv', index=False)
runners

,runner_id,registration_date
0,1,2021-01-03
1,2,2021-01-10
2,3,2021-01-17
3,4,2021-01-24
4,5,2021-01-31
5,6,2021-02-07
6,7,2021-02-14
7,8,2021-02-21
8,9,2021-02-28
9,10,2021-03-07


## Load Orders

This cell loads the orders from CSV first; if not present, it tries XLSX. Adjust file paths if needed.


In [8]:
import os

# Cargar customer_orders y runner_orders desde CSV o XLSX
def load_orders(filename_base):
    csv_path = os.path.join(base_path, filename_base + '.csv')
    xlsx_path = os.path.join(base_path, filename_base + '.xlsx')
    if os.path.exists(csv_path):
        return pd.read_csv(csv_path)
    elif os.path.exists(xlsx_path):
        return pd.read_excel(xlsx_path)
    else:
        raise FileNotFoundError(f"No se encontró {filename_base}.csv ni {filename_base}.xlsx en {base_path}")

## Cleaning & Normalization Helpers

- Parse `distance` (to float km) and `duration` (to minutes).
- Normalize `cancellation` labels (lowercase, strip).
- Enforce FK integrity and logical constraints.


In [9]:

import numpy as np

# Base path in Google Drive


# --- Load small dimension tables ---
pizza_names     = pd.read_csv(base_path +'pizza_names.csv')
pizza_toppings  = pd.read_csv(base_path + 'pizza_toppings.csv')
pizza_recipes   = pd.read_csv(base_path + 'pizza_recipes.csv')
runners         = pd.read_csv(base_path + 'runners.csv')

# --- Load big fact tables ---
customer_orders = pd.read_csv(base_path + 'customer_orders.csv')
runner_orders   = pd.read_csv(base_path + 'runner_orders.csv')

print("pizza_names:", pizza_names.shape)
print("pizza_toppings:", pizza_toppings.shape)
print("pizza_recipes:", pizza_recipes.shape)
print("runners:", runners.shape)
print("customer_orders:", customer_orders.shape)
print("runner_orders:", runner_orders.shape)


pizza_names: (7, 2)
pizza_toppings: (15, 2)
pizza_recipes: (7, 2)
runners: (15, 2)
customer_orders: (2101, 6)
runner_orders: (1500, 6)


In [10]:
customer_orders.head()

,order_id,customer_id,pizza_id,exclusions,extras,order_date
0,2481,500,2,NaN,NaN,2025-02-21 16:06:46
1,1897,269,6,NaN,NaN,2025-08-12 08:27:50
2,1842,59,7,NaN,NaN,2024-12-23 20:50:23
3,1869,710,6,NaN,NaN,2025-01-31 09:16:49
4,2384,189,3,NaN,NaN,2024-11-30 17:35:37




```
# This is formatted as code
```


# /* --------------------
#   Case Study Questions
#   --------------------*/
A. Pizza Metrics

    How many pizzas were ordered?
    How many unique customer orders were made?
    How many successful orders were delivered by each runner?
    How many of each type of pizza was delivered?
    How many Vegetarian and Meatlovers were ordered by each customer?
    What was the maximum number of pizzas delivered in a single order?
    For each customer, how many delivered pizzas had at least 1 change and how many had no changes?
    How many pizzas were delivered that had both exclusions and extras?
    What was the total volume of pizzas ordered for each hour of the day?
    What was the volume of orders for each day of the week?

B. Runner and Customer Experience

    How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)
    What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?
    Is there any relationship between the number of pizzas and how long the order takes to prepare?
    What was the average distance travelled for each customer?
    What was the difference between the longest and shortest delivery times for all orders?
    What was the average speed for each runner for each delivery and do you notice any trend for these values?
    What is the successful delivery percentage for each runner?

🍕 section c — customer & business intelligence

C1. total customer spend
 which customers bring in the most revenue?

use to argue for vip memberships or spend-based rewards.

C2. customer frequency (distinct days of orders)
 who orders regularly vs. one-off customers?

segment into loyal customers vs. occasional customers.

hint at frequency-based discounts (e.g., 5th order free).

C3. first pizza ordered by each customer
 what attracts customers initially?

good to identify entry-point pizzas (the hook item that brings people in).

hint at discounts on first-order pizzas to acquire new customers.

C4. overall best-seller pizza
 which pizza keeps the lights on?

highlight as a flagship product to promote.

use for seasonal bundles (“summer deal with our #1 pizza”).

C5. most popular pizza by customer
 can we personalize offers?

recommend personalized “customer favorites” discounts.

hint toward AI/BI-driven recommender systems.

C6. regulars with ≥30 orders and their go-to pizzas
 who are the heavy hitters and what do they like?

obvious loyalty program candidates.

pitch: “keep them happy with exclusive rewards so they don’t churn.”

C7. customers with very consistent habits (always order the same pizza)
 creatures of habit = stable recurring revenue.

membership idea: “pizza subscription” (weekly plan with their pizza auto-delivered).

C8. the “perfect pair”

great marketing story: “find your pizza soulmate.”

pitch: social media campaign + 2-for-1  perfect pizza couples’ promo.

C9. peak order times
👉 what hours & days matter most?

operational: staff scheduling.

marketing: happy hour discounts in slow periods, premium pricing at peak times.

C10. best candidates for loyalty program
👉 combine spend + frequency + consistency.

identify top 5–10% customers.

suggest tiered memberships: silver/gold/platinum.

seasonal perks: double points in winter when sales slow.

# Entregable

final presentation = a business intelligence pitch deck:

customer segmentation (loyal vs occasional vs perfect pair).

menu insights (flagship pizza, first-order hook, personal favorites).

time insights (peak hours, seasonal discounts).

strategic recommendations:

loyalty program design,

subscription/membership tiers,

seasonal & time-based promos,

“perfect pair” marketing campaign.

In [11]:
import sqlite3

db_path = f"{base_path}dannys_pizza.sqlite"
conn = sqlite3.connect(db_path)
c = conn.cursor()

# drop existing tables (clean slate)
for t in [
    'pizza_names','pizza_toppings','pizza_recipes',
    'runners','customer_orders','runner_orders'
]:
    c.execute(f"DROP TABLE IF EXISTS {t};")

# create empty tables with the canonical column names
c.execute("""CREATE TABLE pizza_names (
  pizza_id INTEGER,
  pizza_name TEXT
);""")

c.execute("""CREATE TABLE pizza_toppings (
  topping_id INTEGER,
  topping_name TEXT
);""")

c.execute("""CREATE TABLE pizza_recipes (
  pizza_id INTEGER,
  toppings TEXT
);""")

c.execute("""CREATE TABLE runners (
  runner_id INTEGER,
  registration_date TEXT
);""")

# NOTE: keep your current column names exactly as they are in the DataFrame
# If your DF uses 'order_date', keep it; if it's 'order_time', keep that.
# Below uses 'order_date'—change to 'order_time' if that’s your DF.
c.execute("""CREATE TABLE customer_orders (
  order_id INTEGER,
  customer_id INTEGER,
  pizza_id INTEGER,
  exclusions TEXT,
  extras TEXT,
  order_date TEXT
);""")

c.execute("""CREATE TABLE runner_orders (
  order_id INTEGER,
  runner_id INTEGER,
  pickup_time TEXT,
  distance TEXT,
  duration TEXT,
  cancellation TEXT
);""")

conn.commit()

# append DataFrames exactly as-is (no cleaning)
pizza_names.to_sql('pizza_names', conn, if_exists='append', index=False)
pizza_toppings.to_sql('pizza_toppings', conn, if_exists='append', index=False)
pizza_recipes.to_sql('pizza_recipes', conn, if_exists='append', index=False)
runners.to_sql('runners', conn, if_exists='append', index=False)
customer_orders.to_sql('customer_orders', conn, if_exists='append', index=False)
runner_orders.to_sql('runner_orders', conn, if_exists='append', index=False)

conn.commit()
print("SQLite ready at:", db_path)


SQLite ready at: c:\Users\Anonimo\OneDrive\Documentos\Python\IA\Week 2\Danny's Pizza\dannys_pizza.sqlite


##### Pizza Metrics

* ¿Cuántas pizzas fueron ordenadas?

In [12]:
pd.read_sql("""
SELECT COUNT(*) AS total_pizzas
FROM customer_orders;
""", conn)


,total_pizzas
0,2101


* ¿Cuántos pedidos únicos de clientes?

In [13]:
pd.read_sql("""
SELECT COUNT(DISTINCT order_id) AS unique_orders
FROM customer_orders;
""", conn)


,unique_orders
0,1500


* ¿Cuántos pedidos exitosos por repartidor?

In [14]:
pd.read_sql("""
SELECT runner_id,
       COUNT(runner_id) AS order_count
FROM runner_orders
WHERE distance IS NOT NULL
  AND TRIM(distance) <> ''
  AND cancellation = ''
GROUP BY runner_id
ORDER BY order_count DESC;
""", conn)


,runner_id,order_count


Notice how our query returned empty results? That is a clue something is off in our filter. We wrote
WHERE distance IS NOT NULL
  AND cancellation = ''''

  but in this dataset, the cancellation column does not only use a blank string to mean no cancellation. Sometimes it has the literal word 'null', sometimes it is NULL (the SQL null value), sometimes different casing (Null, NULL). Because of that, our =  condition excluded almost everything.

In [15]:
  pd.read_sql("""
SELECT DISTINCT TRIM(cancellation) AS cancellation_value,
       COUNT(*) AS n
FROM runner_orders
GROUP BY 1
ORDER BY n DESC;
""", conn)


,cancellation_value,n
0,None,1450
1,address issue,13
2,customer no show,9
3,runner sick,7
4,order late,7
5,custmer_no_sho,7
6,runner_unavailable,5
7,restaurant_cancelled,2


In [16]:
# 3a) Successful orders delivered by each runner (ignore distance presence)
pd.read_sql("""
SELECT runner_id,
       COUNT(*) AS order_count
FROM runner_orders
WHERE COALESCE(TRIM(LOWER(cancellation)),'') IN ('', 'null')
GROUP BY runner_id
ORDER BY order_count DESC;
""", conn)


,runner_id,order_count
0,3,115
1,14,112
2,2,110
3,5,109
4,7,101
5,12,96
6,11,96
7,1,93
8,8,92
9,13,91


* ¿Cuántas de cada tipo de pizza fueron entregadas? 

In [17]:
pd.read_sql("""
    SELECT pizza_id,
           COUNT(*) AS delivered_count
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY pizza_id
    ORDER BY delivered_count DESC;
""", conn)

,pizza_id,delivered_count
0,3,451
1,6,385
2,2,328
3,4,271
4,1,226
5,5,207
6,7,158


* ¿Cuántas pizzas Vegetarian y Meatlovers se pidieron por cliente?

In [18]:
pd.read_sql("""
    SELECT customer_id,
           SUM(CASE WHEN pizza_id=2 THEN 1 ELSE 0 END) AS vegetarian_count,
           SUM(CASE WHEN pizza_id=3 THEN 1 ELSE 0 END) AS meatlovers_count
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id;
""", conn)

,customer_id,vegetarian_count,meatlovers_count
0,1,0,1
1,2,0,1
2,3,0,1
3,4,0,0
4,5,4,7
...,...,...,...
552,791,1,0
553,793,0,1
554,794,0,0
555,796,2,0


* ¿Cuál fue el máximo número de pizzas entregadas en un solo pedido?

In [19]:
pd.read_sql("""
    SELECT order_id,
           COUNT(*) AS pizzas_in_order
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY order_id
    ORDER BY pizzas_in_order DESC
    LIMIT 1;
""", conn)

,order_id,pizzas_in_order
0,2441,4


* Para cada cliente, ¿cuántas pizzas entregadas tuvieron al menos un cambio y cuántas no tuvieron cambios?

In [20]:
pd.read_sql("""
    SELECT customer_id,
           SUM(CASE WHEN (TRIM(exclusions) <> '' OR TRIM(extras) <> '') THEN 1 ELSE 0 END) AS pizzas_with_changes,
           SUM(CASE WHEN (TRIM(exclusions) = '' AND TRIM(extras) = '') THEN 1 ELSE 0 END) AS pizzas_without_changes
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id;
""", conn)

,customer_id,pizzas_with_changes,pizzas_without_changes
0,1,0,0
1,2,2,0
2,3,0,0
3,4,1,0
4,5,12,0
...,...,...,...
552,791,0,0
553,793,1,0
554,794,1,0
555,796,2,0


* ¿Cuántas pizzas entregadas tuvieron tanto exclusiones como extras?

In [21]:
pd.read_sql("""
    SELECT COUNT(*) AS pizzas_with_exclusions_and_extras
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
      AND TRIM(exclusions) <> ''
      AND TRIM(extras) <> '';
""", conn)

,pizzas_with_exclusions_and_extras
0,192


* ¿Cuál fue el volumen total de pizzas ordenadas por cada hora del día?

In [22]:
pd.read_sql("""
    SELECT strftime('%H', order_date) AS hour,
           COUNT(*) AS pizzas_ordered
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY hour
    ORDER BY hour;
""", conn)

,hour,pizzas_ordered
0,00,64
1,01,38
2,02,48
3,03,58
4,04,54
5,05,28
6,06,72
7,07,32
8,08,43
9,09,64


* ¿Cuál fue el volumen de pedidos por cada día de la semana?

In [23]:
pd.read_sql("""
    SELECT strftime('%w', order_date) AS weekday,
           COUNT(*) AS pizzas_ordered
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY weekday
    ORDER BY weekday;
""", conn)

,weekday,pizzas_ordered
0,0,426
1,1,167
2,2,168
3,3,180
4,4,241
5,5,401
6,6,443


#### Runner and Customer Experience

1. How many runners signed up for each 1 week period? (week starts 2021-01-01)

In [24]:
pd.read_sql("""
    SELECT strftime('%Y-%W', registration_date) AS week,
           COUNT(*) AS runners_signed_up
    FROM runners
    GROUP BY week
    ORDER BY week;
""", conn)

,week,runners_signed_up
0,2021-00,1
1,2021-01,1
2,2021-02,1
3,2021-03,1
4,2021-04,1
5,2021-05,1
6,2021-06,1
7,2021-07,1
8,2021-08,1
9,2021-09,1


2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [25]:
pd.read_sql("""
    SELECT runner_id,
           AVG(CAST(strftime('%s', pickup_time) - strftime('%s', order_date) AS REAL)/60.0) AS avg_arrival_minutes
    FROM runner_orders
    JOIN customer_orders USING(order_id)
    WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
      AND pickup_time IS NOT NULL
      AND order_date IS NOT NULL
    GROUP BY runner_id;
""", conn)


,runner_id,avg_arrival_minutes
0,1,13.142857
1,2,13.606897
2,3,13.341935
3,4,13.703704
4,5,13.790850
5,6,14.313043
6,7,12.977273
7,8,12.876033
8,9,13.500000
9,10,13.367521


3. Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [26]:
pd.read_sql("""
    SELECT co.order_id,
           COUNT(*) AS pizzas_in_order,
           ro.duration
    FROM customer_orders co
    JOIN runner_orders ro ON co.order_id = ro.order_id
    WHERE COALESCE(TRIM(LOWER(ro.cancellation)), '') IN ('', 'null')
      AND ro.duration IS NOT NULL
    GROUP BY co.order_id, ro.duration
    ORDER BY pizzas_in_order DESC;
""", conn)


,order_id,pizzas_in_order,duration
0,1106,4,19 minutes
1,1204,4,16 minutes
2,1331,4,15 minutes
3,1401,4,8 minutes
4,1752,4,17 minutes
...,...,...,...
1390,2492,1,7 minutes
1391,2493,1,14 minutes
1392,2494,1,29 minutes
1393,2495,1,18 minutes


#### Customer & business intelligence

C1. Total customer spend

In [27]:
pd.read_sql("""
    SELECT customer_id,
           COUNT(*) AS pizzas_ordered
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id
    ORDER BY pizzas_ordered DESC;
""", conn)

,customer_id,pizzas_ordered
0,620,52
1,30,52
2,717,51
3,657,51
4,500,50
...,...,...
552,10,1
553,8,1
554,7,1
555,4,1


C2. Customer frequency (distinct days of orders)

In [28]:
pd.read_sql("""
    SELECT customer_id,
           COUNT(DISTINCT DATE(order_date)) AS order_days
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id
    ORDER BY order_days DESC;
""", conn)

,customer_id,order_days
0,717,35
1,30,35
2,730,33
3,657,32
4,620,32
...,...,...
552,8,1
553,7,1
554,4,1
555,3,1


C3. First pizza ordered by each customer

In [29]:
pd.read_sql("""
    SELECT customer_id,
           MIN(order_date) AS first_order_date,
           pizza_id
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id
    ORDER BY first_order_date;
""", conn)

,customer_id,first_order_date,pizza_id
0,500,2024-11-01 04:07:33,5
1,464,2024-11-01 14:29:31,6
2,319,2024-11-01 16:21:41,3
3,708,2024-11-01 18:28:31,2
4,438,2024-11-01 19:02:29,6
...,...,...,...
552,296,2025-08-09 18:17:37,6
553,401,2025-08-10 16:39:24,3
554,24,2025-08-10 22:29:27,3
555,74,2025-08-14 00:15:44,5


C4. Overall best-seller pizza

In [30]:
pd.read_sql("""
    SELECT pn.pizza_id,
           pn.pizza_name,
           COUNT(*) AS total_sold
    FROM customer_orders co
    JOIN pizza_names pn ON co.pizza_id = pn.pizza_id
    WHERE co.order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY pn.pizza_id, pn.pizza_name
    ORDER BY total_sold DESC;
""", conn)

,pizza_id,pizza_name,total_sold
0,3,Meat Lovers,451
1,6,Pepperoni,385
2,2,Vegetarian,328
3,4,BBQ Chicken,271
4,1,Margherita,226
5,5,Hawaiian,207
6,7,Vegan Veggie,158


C5. Most popular pizza by customer

In [31]:
pd.read_sql("""
    SELECT customer_id,
           pizza_id,
           COUNT(*) AS times_ordered
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id, pizza_id
    ORDER BY customer_id, times_ordered DESC;
""", conn)

,customer_id,pizza_id,times_ordered
0,1,6,1
1,1,4,1
2,1,3,1
3,2,3,1
4,2,1,1
...,...,...,...
1185,793,3,1
1186,794,6,2
1187,796,2,2
1188,796,1,1


C6. Regulars with ≥30 orders and their go-to pizzas

In [32]:
pd.read_sql("""
    SELECT customer_id,
           pizza_id,
           COUNT(*) AS pizza_count
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id, pizza_id
    HAVING customer_id IN (
        SELECT customer_id
        FROM customer_orders
        WHERE order_id IN (
            SELECT order_id
            FROM runner_orders
            WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
        )
        GROUP BY customer_id
        HAVING COUNT(*) >= 30
    )
      AND COUNT(*) = (
        SELECT MAX(cnt)
        FROM (
            SELECT customer_id, pizza_id, COUNT(*) AS cnt
            FROM customer_orders
            WHERE order_id IN (
                SELECT order_id
                FROM runner_orders
                WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
            )
            GROUP BY customer_id, pizza_id
        )
        WHERE customer_id = customer_orders.customer_id
    )
    ORDER BY pizza_count DESC;
""", conn)

,customer_id,pizza_id,pizza_count
0,657,3,18
1,30,6,15
2,180,3,15
3,620,6,14
4,717,6,13
5,500,3,12
6,730,2,12
7,45,6,11
8,228,2,11
9,240,6,11


C7. Customers with very consistent habits

In [33]:
pd.read_sql("""
    SELECT customer_id,
           COUNT(DISTINCT pizza_id) AS unique_pizzas,
           COUNT(*) AS total_orders
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id
    HAVING unique_pizzas = 1 AND total_orders > 1
    ORDER BY total_orders DESC;
""", conn)

,customer_id,unique_pizzas,total_orders
0,94,1,3
1,794,1,2
2,715,1,2
3,697,1,2
4,537,1,2
5,526,1,2
6,522,1,2
7,521,1,2
8,485,1,2
9,464,1,2


C8. “Perfect pair” — Find pairs of customers who always order together and the same pizza

In [34]:
pd.read_sql("""
    SELECT c1.customer_id AS customer1,
           c2.customer_id AS customer2,
           COUNT(*) AS shared_orders
    FROM customer_orders c1
    JOIN customer_orders c2
      ON c1.order_id = c2.order_id
     AND c1.customer_id < c2.customer_id
     AND c1.pizza_id = c2.pizza_id
    WHERE c1.order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer1, customer2
    HAVING COUNT(*) = (
        SELECT COUNT(*)
        FROM customer_orders
        WHERE customer_id = customer1
          AND order_id IN (
              SELECT order_id
              FROM runner_orders
              WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
          )
    )
       AND COUNT(*) = (
        SELECT COUNT(*)
        FROM customer_orders
        WHERE customer_id = customer2
          AND order_id IN (
              SELECT order_id
              FROM runner_orders
              WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
          )
    )
    ORDER BY shared_orders DESC;
""", conn)

,customer1,customer2,shared_orders


In [35]:
pd.read_sql("""
    SELECT
        c1.customer_id AS customer1,
        c2.customer_id AS customer2,
        c1.pizza_id,
        pn.pizza_name,
        COUNT(*) AS similar_pizzas
    FROM customer_orders c1
    JOIN customer_orders c2
      ON c1.pizza_id = c2.pizza_id
     AND c1.customer_id < c2.customer_id
    JOIN pizza_names pn ON c1.pizza_id = pn.pizza_id
    WHERE c1.order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
      AND c2.order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer1, customer2, c1.pizza_id, pn.pizza_name
    HAVING similar_pizzas >= 2
    ORDER BY similar_pizzas DESC
    LIMIT 15;
""", conn)

,customer1,customer2,pizza_id,pizza_name,similar_pizzas
0,180,657,3,Meat Lovers,270
1,500,657,3,Meat Lovers,216
2,657,717,3,Meat Lovers,216
3,30,620,6,Pepperoni,210
4,30,717,6,Pepperoni,195
5,620,717,6,Pepperoni,182
6,180,500,3,Meat Lovers,180
7,180,717,3,Meat Lovers,180
8,657,667,3,Meat Lovers,180
9,30,45,6,Pepperoni,165


C9. Peak order times

In [36]:
pd.read_sql("""
    SELECT strftime('%w', order_date) AS weekday,
           strftime('%H', order_date) AS hour,
           COUNT(*) AS pizzas_ordered
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY weekday, hour
    ORDER BY weekday, hour;
""", conn)

,weekday,hour,pizzas_ordered
0,0,00,12
1,0,01,7
2,0,02,1
3,0,03,18
4,0,04,7
...,...,...,...
159,6,19,66
160,6,20,28
161,6,21,20
162,6,22,8


C10. Best candidates for loyalty program

In [37]:
pd.read_sql("""
    SELECT customer_id,
           COUNT(*) AS total_orders,
           COUNT(DISTINCT DATE(order_date)) AS order_days,
           COUNT(DISTINCT pizza_id) AS unique_pizzas
    FROM customer_orders
    WHERE order_id IN (
        SELECT order_id
        FROM runner_orders
        WHERE COALESCE(TRIM(LOWER(cancellation)), '') IN ('', 'null')
    )
    GROUP BY customer_id
    ORDER BY total_orders DESC, order_days DESC, unique_pizzas ASC
    LIMIT 10;
""", conn)

,customer_id,total_orders,order_days,unique_pizzas
0,30,52,35,7
1,620,52,32,7
2,717,51,35,7
3,657,51,32,7
4,500,50,32,7
5,730,49,33,7
6,180,49,30,7
7,240,47,30,7
8,547,46,29,7
9,755,44,30,7
